# **Algoritmo de Extração de Dados de Royalties - Aracaju/SE**

O presente projeto visa extrair os dados do portal de transparência da prefeitura de Aracaju, especificamente dados onde a fonte de arrecadação são royalties do petróleo.

O projeto utiliza as seguintes ferramentas
- Automação de navegação e extração dos dados:
    - Selenium;
    - BeatifulSoup;
- Análise Exploratória dos Dados (EDA):
    - Pandas
    - Matplotlib

## 1. Importando bibliotecas necessárias e Definindo constantes

In [ ]:
# ==============================================================================
# 1. IMPORTAÇÕES E CONFIGURAÇÕES GLOBAIS
# ==============================================================================

import os
import re
import sys
import time
import csv
import glob
import unicodedata
import logging
from typing import List, Dict, Optional
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import partial

import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    StaleElementReferenceException,
    ElementClickInterceptedException
)
from webdriver_manager.chrome import ChromeDriverManager


class TaskIdFilter(logging.Filter):
    """
    Filtro customizado para adicionar um atributo 'task_id' aos registros de log,
    permitindo identificar a origem do log por thread ou tarefa.

    Args:
        task_id (str): Identificador da thread ou tarefa para associar aos logs.
    """
    def __init__(self, task_id: str = 'MainThread'):
        super().__init__()
        self.task_id = task_id

    def filter(self, record: logging.LogRecord) -> bool:
        """
        Adiciona o atributo 'task_id' ao registro de log, se ainda não existir.

        Args:
            record (logging.LogRecord): Registro de log a ser filtrado.

        Returns:
            bool: Sempre True para que o registro não seja filtrado.
        """
        if not hasattr(record, 'task_id'):
            record.task_id = self.task_id
        return True


def setup_logging(log_file: str = '../logs/aracaju/2020-2024_osr_aracaju.log',
                  task_id: str = 'MainThread') -> logging.Logger:
    """
    Configura o logger principal da aplicação para registrar mensagens de log em
    arquivo e no console, formatando as mensagens e adicionando um filtro para
    identificação por tarefa/thread.

    O logger é configurado para o nível INFO e acima. Também ajusta níveis de
    log de bibliotecas externas para WARNING, reduzindo ruído.

    Parâmetros:
        log_file (str): Caminho do arquivo de log a ser criado ou sobrescrito.
        task_id (str): Identificador a ser inserido em cada mensagem de log,
                       útil para distinguir logs de threads/tarefas.

    Retorna:
        logging.Logger: Instância do logger configurado, pronta para uso.

    Exemplo de uso:
        logger = setup_logging()
        logger.info("Logging configurado com sucesso.")
    """
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    log_format = "[%(task_id)s] - %(asctime)s - %(levelname)s - %(message)s"

    logger = logging.getLogger('osr_barra')
    logger.setLevel(logging.INFO)

    if logger.hasHandlers():
        logger.handlers.clear()

    formatter = logging.Formatter(log_format)

    file_handler = logging.FileHandler(log_file, mode='w', encoding='utf-8')
    file_handler.setFormatter(formatter)
    file_handler.addFilter(TaskIdFilter(task_id))

    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    console_handler.addFilter(TaskIdFilter(task_id))

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    logging.getLogger('selenium').setLevel(logging.WARNING)
    logging.getLogger('urllib3').setLevel(logging.WARNING)
    logging.getLogger('WDM').setLevel(logging.WARNING)

    return logger


logger = setup_logging()
logger.info("Logging configurado com sucesso.")

In [ ]:
# ==============================================================================
# 2. CONSTANTES
# ==============================================================================

# # Lista de anos disponíveis para scraping
# ANOS = [str(ano) for ano in range(2014, 2026)]

# # Lista de meses com dois dígitos (01 a 12)
# MESES = [f"{m:02d}" for m in range(1, 13)]

# Colunas principais fixas do dataframe gerado. 
# As colunas de detalhe (por exemplo, Histórico) são adicionadas dinamicamente após o parsing.
COLUNAS_PRINCIPAIS = [
    'orgao',       # Órgão responsável
    'unidade',     # Unidade gestora
    'data',        # Data da transação
    'empenho',     # Número do empenho
    'processo',    # Número do processo
    'credor',      # Nome do credor
    'cpf_cnpj',    # Documento do credor
    'pago',        # Valor pago
    'retido',      # Valor retido
    'anulacao'     # Valor anulado
]

# # Diretório onde os arquivos CSV serão salvos
# OUTPUT_DIR = 'dados_royalties_aracaju'
# os.makedirs(OUTPUT_DIR, exist_ok=True)  # Cria o diretório se não existir

# Termos e códigos associados a royalties, usados para filtragem dos dados extraídos
TERMOS_ROYALTIES = [
    'royalty', 'royalties', 'petroleo',
    '15300000',  # Transferência da União Referente a Royalties do Petróleo
    '15400000',  # Transferência dos Estados Referente a Royalties do Petróleo
    '17050000',  # Transferência dos estados referente a royalties do petróleo e gás natural
    '17200000',  # Transferências da União Referente a Royalties do Petróleo e Gás Natural
    '17210000',  # Transferências a União Referentes a Cessão Onerosa de Petróleo - Lei nº 13.885/2019
    '0120000'    # Royalties - Petróleo, Xistos e Gás (exemplo típico: 09/2015)
]


## 2. Definindo Funções Auxiliares

- **normalizar(texto)**:
    - Esta função é uma utilitária que pode ser usada por várias outras funções, então deve ser definida bem no início.
- **start_driver(headless=False, url=str)**:
    - Inicializa o navegador. É uma função fundamental e não depende de nenhuma das outras funções específicas de extração.
- **wait_for_page_load(driver, timeout=10)**:
    - Uma função de espera genérica.
- **wait_for_loading_to_disappear(driver, timeout=10)**:
    - Espera o gif de carregamento desaparecer
- **selecionar_pagamentos(driver)**:
    - Navega para a aba de pagamentos. Depende apenas do driver já iniciado.
- **selecionar_ano_mes(driver, ano: str, mes: str)**:
    - Seleciona o ano e mês. Depende do driver e da página de pagamentos.

In [ ]:
RE_REMOVE_PUNCTUATION = re.compile(r'[^a-zA-Z0-9\s]')

def normalizar(texto):
    """
    Normaliza um texto removendo acentos, pontuações e convertendo para minúsculas.

    Args:
        texto (str): Texto original a ser normalizado.

    Returns:
        str: Texto normalizado (sem acentos, pontuação e em letras minúsculas).
    """
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = RE_REMOVE_PUNCTUATION.sub('', texto)
    return texto.lower()

def start_driver(headless=False, url=str):
  """
  Inicializa o driver do Chrome com ou sem modo headless,
  acessando a URL da transparência ou do portal cidadão.

  Args:
      headless (bool): Se True, inicia o navegador em modo headless.
      url (str): "transparencia" ou "portal_(cidade)".

  Returns:
      webdriver.Chrome: Instância do ChromeDriver configurada.
  
  Raises:
        ValueError: Se o valor de 'url' não for reconhecido.
  """
  
  logger.info(" Iniciando driver do Chrome...\n")
     
  options = webdriver.ChromeOptions()
  if headless:
    options.add_argument("--headless") # Headless = Não abrir a janela do navegador
    options.add_argument('--disable-gpu') # desativa o uso da GPU
    options.add_argument('--no-sandbox') # desativa o sandbox (ambiente de execução isolado)
  else:
    options.add_argument('--window-size=1920,1080')  # Recomendo adicionar tamanho fixo
    options.add_argument("--disable-blink-features=AutomationControlled") # Desativa os recursos do Blink (motor do Chrome) que identificam automação
    options.add_experimental_option("excludeSwitches", ["enable-automation"]) # Remove a mensagem "Chrome está sendo controlado por software automatizado"
    
  # Forma recomendada de configurar o WebDriver, pois o DriveManager lida com compatibildiade de versões
  driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
  
  # Dicionário de URLs possíveis
  urls = {
    # Transparência > Despesas > #despesas 
    "transparencia": "https://transparencia.aracaju.se.gov.br/prefeitura/despesas-2/",
    # Municipio Online > Cidadão > Plan. e Exec. Orçamentaria > Despesa
    "portal_aracaju": "https://www.municipioonline.com.br/se/prefeitura/aracaju/cidadao/despesa", # (2014 - 2025)
    "portal_pacatuba": "https://www.municipioonline.com.br/se/prefeitura/pacatuba/cidadao/despesa", # (Não há pagamentos listados)
    "portal_barra": "https://www.municipioonline.com.br/se/prefeitura/barradoscoqueiros/cidadao/despesa", # (1999)(2018 - 2025)
    "portal_pirambu": "https://www.municipioonline.com.br/se/prefeitura/pirambu/cidadao/despesa" # (2016 - 2025)
  }
  
  if url not in urls:
    raise ValueError("Parâmetro 'url' deve ser 'transparencia' ou 'portal_(cidade)'.")
  
  driver.get(urls[url])
  #logger.info(f" Acessada URL: {urls[url]}\n")
  return driver

def wait_for_page_load(driver, timeout=10):
  """
  Aguarda até que a página carregue completamente, verificando a presença do elemento <body>.

  Args:
    driver (webdriver.Chrome): Instância do navegador.
    timeout (int): Tempo máximo de espera em segundos.
  """
  
  wait = WebDriverWait(driver, timeout)
  try:
    wait.until(EC.visibility_of_element_located((By.TAG_NAME, 'body')))
    logger.info(" Página carregada com sucesso.\n")
  except Exception as e:
    logger.error(f" Erro ao esperar carregamento da página: {e}\n")
    

def wait_for_loading_to_disappear(driver, timeout=60, id="loading"): # Aumentei o timeout padrão para 60s
    """
    Aguarda até que o indicador de carregamento com um ID específico desapareça da tela.

    Args:
        driver (webdriver.Chrome): Instância do navegador.
        timeout (int): Tempo máximo de espera (em segundos).
        id (str): ID do elemento de carregamento (default: "loading").

    Raises:
        TimeoutException: (opcional, comentada) Se o elemento não desaparecer no tempo limite.
    """
    try:
        logger.debug(" Aguardando o indicador de carregamento (ID 'loading') desaparecer...")
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((By.ID, id))
        )
        logger.debug(" Indicador de carregamento (ID 'loading') desapareceu.")
    except TimeoutException:
        # Se o timeout for atingido, logamos um aviso.
        # Dependendo da criticidade, você pode querer lançar a exceção novamente.
        logger.warning(f" Timeout: Indicador de carregamento (ID 'loading') não desapareceu em {timeout} segundos.")
        # Descomente a linha abaixo se for uma falha crítica:
        # raise    
    
    
def selecionar_pagamentos(driver):
    """
    Navega até a aba de pagamentos.

    Etapas:
        1. (Exclusivo para URL de Transparencia) Aguarda o carregamento e troca para o iframe de ID "dados".
        2. Clica no quarto item do menu (Pagamentos).
        3. Aguarda o carregamento da nova página.

    Args:
        driver (webdriver.Chrome): Instância do navegador com a página já carregada.
        
    Raises:
        Exception: Em caso de falha na navegação.
    """
    
    try:
        # 1. Mudar para o iframe "dados" (caso esteja no "Transparência")
        # WebDriverWait(driver, 10).until(
        #     EC.frame_to_be_available_and_switch_to_it((By.ID, "dados"))
        # )
        
        # 2. Clicar no quarto item da lista (Pagamentos)
        elemento = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//ul/li[4]/a"))
        )
        elemento.click()
    
        # 3. (Opcional) Voltar para o conteúdo principal
        #driver.switch_to.default_content()
        
        wait_for_loading_to_disappear(driver)
        
        logger.info("Aba Pagamentos selecionada com sucesso.\n")  
        
    except Exception as e:
        logger.error(f"Erro ao selecionar aba de Pagamentos: {e}")

def selecionar_ano_mes(driver, ano: str, mes: str):
    """
    Seleciona um ano e mês nos filtros da aba Pagamentos e aplica o filtro.

    Args:
        driver (webdriver.Chrome): Instância do navegador.
        ano (str): Ano a ser selecionado (ex: "2023").
        mes (str): Mês a ser selecionado (ex: "04" para abril).

    Raises:
        Exception: Caso ocorra erro ao aplicar o filtro.
    """
    # Espera carregar
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "dataTables-Pagamentos")))
    
    try:
        # Seleciona ano
        select_ano = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ddlAnoPagamentos")))
        Select(select_ano).select_by_value(ano)
        
        # Seleciona mês
        select_mes = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ddlMesPagamentos")))
        Select(select_mes).select_by_value(mes)
        
        # Aplica filtro
        btn_filtrar = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "btnFiltrarPagamentos")))
        btn_filtrar.click()
        
        # Espera carregar
        if WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "dataTables-Pagamentos"))):
        
            wait_for_loading_to_disappear(driver)
            logger.info(f"Filtro para {mes}/{ano} aplicado e carregamento concluído.")
        
    except Exception as e:
        logger.error(f"Erro ao selecionar {mes}/{ano}: {str(e)}")
        raise


## 3. Definindo Funções de Extração de Dados

- **extrair_dados_pagamentos(driver) -> pd.DataFrame**:
    - Esta é a função central que faz o scraping da tabela principal e dos detalhes. Ela dependerá de normalizar() e das operações básicas do driver.

In [ ]:
def extrair_dados_pagamentos(driver) -> pd.DataFrame:
    """
    Extrai dados de pagamentos relacionados a royalties a partir de uma tabela interativa com detalhes ocultos.

    A função percorre cada linha da tabela de pagamentos, expande os detalhes,
    verifica se a 'Fonte de Recurso' contém termos relacionados a royalties e,
    se confirmado, extrai tanto os dados principais quanto os dados detalhados.

    Args:
        driver (selenium.webdriver): Instância do WebDriver já posicionada na página desejada.

    Returns:
        pd.DataFrame: DataFrame contendo os dados de pagamentos com fonte de recurso relacionada a royalties.
    """
    dados_completos = []
    cont_is_royalty = 0
    
    # --- Espera a presença da tabela de pagamentos na página ---
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.ID, "dataTables-Pagamentos"))
        )
        
        xpath_base_linhas_principais = (
            "//table[@id='dataTables-Pagamentos']/tbody/tr[@role='row'][contains(@class, 'odd') or contains(@class, 'even')]"
        )
        
        try:
            linhas_na_pagina_atual = driver.find_elements(By.XPATH, xpath_base_linhas_principais)
            num_linhas_principais = len(linhas_na_pagina_atual)
            if num_linhas_principais == 0:
                logger.info("Nenhuma linha principal de dados encontrada nesta página.")
                return pd.DataFrame()
            logger.info(f"Encontradas {num_linhas_principais} linhas principais para processar.")
        except Exception as e_count:
            logger.error(f"Erro ao contar linhas principais: {e_count}")
            return pd.DataFrame()

        for i in tqdm(range(num_linhas_principais), desc="Processando linhas da tabela\n"):
            logger.debug(f"Iniciando processamento da linha índice {i}.")
            current_row_xpath = f"({xpath_base_linhas_principais})[{i+1}]"
            linha_principal_element = None
            details_opened_successfully = False
            
            try:
                # Etapa 0: Localizar a linha principal da iteração atual
                linha_principal_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, current_row_xpath))
                )
                logger.debug(f"Linha {i}: Linha principal '{current_row_xpath}' localizada.")

                # Etapa 1: Abrir Detalhes
                btn_detalhes = linha_principal_element.find_element(By.XPATH, "./td[1][contains(@class, 'details-control')]")
                
                # Verifica se já está aberto antes de clicar
                if "shown" not in linha_principal_element.get_attribute("class"):
                    logger.debug(f"Linha {i}: Tentando abrir detalhes.")
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", btn_detalhes)
                    time.sleep(0.3)
                    btn_detalhes.click()
                    time.sleep(1)
                    
                    # melhorar espera dos detalhes
                    # WebDriverWait(driver, 10).until(
                    #     EC.element_to_be_clickable((By.XPATH, details_table_responsive_xpath)))
                    
                    WebDriverWait(driver, 10).until(
                        lambda d: "shown" in d.find_element(By.XPATH, current_row_xpath).get_attribute("class")
                    )
                    logger.debug(f"Linha {i}: Detalhes abertos, classe 'shown' confirmada.")
                    # Se houver um loader GERAL após a expansão de detalhes:
                    # wait_for_loading_to_disappear(driver, timeout=10)
                else:
                    logger.debug(f"Linha {i}: Detalhes já estavam abertos (classe 'shown' presente).")
                
                details_opened_successfully = True # Marca que conseguimos (ou já estava) abrir
                time.sleep(5) # Pequena pausa para estabilização do JS após abrir/confirmar aberto

                # --- NOVA ESPERA: Aguardar o div.table-responsive ficar visível ---
                # Esta div está dentro da linha de detalhes (following-sibling da linha principal)
                details_content_wrapper_xpath = f"{current_row_xpath}/following-sibling::tr[1]"
                details_table_responsive_xpath = f"{details_content_wrapper_xpath}//div[@class='table-responsive']"
                
                logger.debug(f"Linha {i}: Aguardando 'div.table-responsive' em '{details_table_responsive_xpath}' ficar visível.")
                WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, details_table_responsive_xpath))
                )
                logger.debug(f"Linha {i}: 'div.table-responsive' dos detalhes está visível.")
                
                # Obter o container da linha de detalhes (a <tr>)
                linha_detalhes_container = driver.find_element(By.XPATH, details_content_wrapper_xpath)


                # Etapa 2: Extrair APENAS "Fonte de Recurso" dos detalhes
                fonte_recurso_valor = None
                dados_detalhes_preview = {} # Para armazenar temporariamente todos os detalhes lidos na primeira passada

                logger.debug(f"Linha {i}: Re-confirmando linha principal '{current_row_xpath}' para ler detalhes.")
                linha_principal_refrescada_para_leitura = WebDriverWait(driver, 7).until(
                    EC.visibility_of_element_located((By.XPATH, current_row_xpath))
                )
                
                
                xpath_relativo_container_detalhes = "./following-sibling::tr[1]"
                linha_detalhes_container = WebDriverWait(linha_principal_refrescada_para_leitura, 10).until(
                    EC.visibility_of_element_located((By.XPATH, xpath_relativo_container_detalhes))
                )
                
                tabela_detalhes_interna = linha_detalhes_container.find_element(By.XPATH, ".//div[@class='table-responsive']/table")
                linhas_da_tabela_detalhes = tabela_detalhes_interna.find_elements(By.XPATH, "./tbody/tr")

                for linha_det_interna in linhas_da_tabela_detalhes:
                    try:
                        chave_el = linha_det_interna.find_element(By.XPATH, "./th")
                        valor_el = linha_det_interna.find_element(By.XPATH, "./td")
                        chave_bruta = chave_el.text.strip()
                        chave_limpa = chave_bruta.replace(":", "").replace(u'\xa0', ' ').strip()
                        valor_limpo = valor_el.text.strip()
                        chave_norm = normalizar(chave_limpa).replace(" ", "_")
                        
                        if chave_norm: # Armazena todos os detalhes lidos nesta primeira passada
                            dados_detalhes_preview[chave_norm] = valor_limpo
                        
                        if chave_norm == "fonte_de_recurso":
                            fonte_recurso_valor = valor_limpo
                            logger.debug(f"Linha {i}: 'Fonte de Recurso' encontrada: '{fonte_recurso_valor}'")
                            # Não damos break aqui, lemos todos os detalhes uma vez.
                    except NoSuchElementException:
                        continue # Pula linhas de detalhe malformadas
                    except Exception as e_det_item:
                        logger.warning(f"Linha {i}: Erro ao ler item de detalhe ({chave_bruta if 'chave_bruta' in locals() else 'N/A'}): {e_det_item}")

                logger.debug(f"Linha {i}: Detalhes da tabela interna lidos. 'Fonte de Recurso': '{fonte_recurso_valor}'")

                # Extrair Histórico Empenho (relativo a linha_detalhes_container)
                try:
                    hist_empenho_el = linha_detalhes_container.find_element(By.XPATH, ".//div[contains(@class, 'panel-heading') and normalize-space(.)='Histórico Empenho']/following-sibling::div[contains(@class, 'panel-body')]/p")
                    dados_detalhes_preview['historico_empenho'] = hist_empenho_el.text.strip()
                    logger.debug(f"Linha {i}: Histórico Empenho: '{dados_detalhes_preview['historico_empenho']}'")
                except NoSuchElementException: logger.debug(f"Linha {i}: Histórico Empenho não encontrado.")
                except Exception as e_he: logger.warning(f"Linha {i}: Erro ao extrair Histórico Empenho: {e_he}")

                # Extrair Histórico Pagamento (relativo a linha_detalhes_container)
                try:
                    hist_pagamento_el = linha_detalhes_container.find_element(By.XPATH, ".//div[contains(@class, 'panel-heading') and normalize-space(.)='Histórico Pagamento']/following-sibling::div[contains(@class, 'panel-body')]/p")
                    dados_detalhes_preview['historico_pagamento'] = hist_pagamento_el.text.strip()
                    logger.debug(f"Linha {i}: Histórico Pagamento: '{dados_detalhes_preview['historico_pagamento']}'")
                except NoSuchElementException: logger.debug(f"Linha {i}: Histórico Pagamento não encontrado.")
                except Exception as e_hp: logger.warning(f"Linha {i}: Erro ao extrair Histórico Pagamento: {e_hp}")


                if not fonte_recurso_valor and dados_detalhes_preview: # Se lemos detalhes mas não achamos a fonte
                     logger.warning(f"Linha {i}: 'Fonte de Recurso' não encontrada nos detalhes, embora detalhes tenham sido lidos.")

                
                # Etapa 3: Verificar se é Royalties
                is_royalty_related = False
                if fonte_recurso_valor:
                    fonte_norm_check = normalizar(fonte_recurso_valor)
                    if any(termo in fonte_norm_check for termo in TERMOS_ROYALTIES ):
                        is_royalty_related = True
                        cont_is_royalty += 1
                        
                
                # Etapa 4: Se for Royalties, extrair dados da linha principal e combinar
                if is_royalty_related:
                    logger.info(f"Linha {i}: Royalties detectados (Fonte: '{fonte_recurso_valor}'). Extraindo dados completos.")
                    dados_linha = {} # Inicia o dicionário para esta linha
                    
                    # Re-localizar linha_principal_element para garantir que está fresco antes de pegar suas células
                    linha_principal_element = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, current_row_xpath))
                    )
                    celulas_lp = linha_principal_element.find_elements(By.XPATH, "./td")
                    if len(celulas_lp) > 10:
                        dados_linha['orgao'] = celulas_lp[1].text
                        dados_linha['unidade'] = celulas_lp[2].text
                        dados_linha['data'] = celulas_lp[3].text
                        dados_linha['empenho'] = celulas_lp[4].text
                        dados_linha['processo'] = celulas_lp[5].text
                        dados_linha['credor'] = celulas_lp[6].text
                        dados_linha['cpf_cnpj'] = celulas_lp[7].text
                        dados_linha['pago'] = celulas_lp[8].text
                        dados_linha['retido'] = celulas_lp[9].text
                        dados_linha['anulacao'] = celulas_lp[10].text
                    else:
                        logger.error(f"Linha {i}: Falha ao re-ler células da linha principal para item de royalty. Pulando item.")
                        # Ir para a lógica de fechar detalhes e continuar
                        details_opened_successfully = True # Força a tentativa de fechar
                        is_royalty_related = False # Evita adicionar dados incompletos
                        # Pula para o bloco finally da linha para fechar os detalhes

                    # Adicionar os detalhes já lidos (dados_detalhes_preview)
                    dados_linha.update(dados_detalhes_preview)
                    dados_completos.append(dados_linha)
                    
                     
                else:
                    logger.debug(f"Linha {i}: Não é de royalties (Fonte: '{fonte_recurso_valor if fonte_recurso_valor else 'Não encontrada/lida'}'). Pulando extração completa.")

            except StaleElementReferenceException as sere:
                logger.error(f"Linha {i}: StaleElementReferenceException DURANTE o processamento - {str(sere)}. Pulando linha.")
                details_opened_successfully = False # Não podemos garantir estado para fechar
                continue # Pula para a próxima linha no loop 'for i'
            except TimeoutException as te:
                logger.error(f"Linha {i}: TimeoutException DURANTE o processamento - {str(te)}. Pulando linha.")
                details_opened_successfully = False
                continue
            except Exception as e_linha_proc:
                logger.error(f"Linha {i}: Erro INESPERADO DURANTE o processamento - {str(e_linha_proc)}. Pulando linha.")
                details_opened_successfully = False
                continue
            
            # Etapa 5: Fechar Detalhes (SEMPRE tenta fechar se foram abertos ou estavam abertos)
            if details_opened_successfully: # Tenta fechar apenas se a abertura foi (ou parecia) bem-sucedida
                try:
                    logger.debug(f"Linha {i}: Tentando fechar detalhes (estado 'details_opened_successfully': {details_opened_successfully}).")
                    # Re-localiza a linha principal antes de checar 'shown' e fechar
                    linha_atual_para_fechar = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, current_row_xpath))
                    )
                    if "shown" in linha_atual_para_fechar.get_attribute("class"):
                        btn_detalhes_fechar = linha_atual_para_fechar.find_element(By.XPATH, "./td[1][contains(@class, 'details-control')]")
                        driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", btn_detalhes_fechar)
                        time.sleep(0.3)
                        btn_detalhes_fechar.click()
                        WebDriverWait(driver, 10).until(
                            lambda d: "shown" not in d.find_element(By.XPATH, current_row_xpath).get_attribute("class")
                        )
                        logger.debug(f"Linha {i}: Detalhes fechados.")
                    else:
                        logger.debug(f"Linha {i}: Detalhes já estavam fechados (classe 'shown' não encontrada ao tentar fechar).")
                except Exception as e_close:
                    logger.error(f"Linha {i}: Erro ao tentar fechar detalhes - {str(e_close)}. Continuando para a próxima linha.")
            
            logger.info(f"\nLinha {i}: Fim do processamento da linha.")

        return pd.DataFrame(dados_completos)

    except Exception as e_geral:
        logger.critical(f"Erro GERAL e FATAL na função extrair_dados_pagamentos: {str(e_geral)}")
        return pd.DataFrame()

## 4. Função de Paginação

- **ir_para_proxima_pagina(driver) -> bool**:
    - Precisa do driver para interagir com os elementos da página.

In [ ]:
def ir_para_proxima_pagina(driver) -> bool:
    """
    Tenta clicar no botão da próxima página na tabela de pagamentos.
    Inclui alternativas de clique para maior robustez.
    Retorna True se conseguiu ir para a próxima página, False caso contrário.
    """
    try:
        proxima_pagina_li_locator = (By.ID, "dataTables-Pagamentos_next")

        # Espera que o elemento <li> esteja presente e seja minimamente interativo
        logger.debug("Aguardando o <li> do botão 'Próximo' ser localizável.")
        proxima_pagina_li_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(proxima_pagina_li_locator)
        )
        
        # Verifica se o botão <li> está desabilitado pela classe
        if "disabled" in proxima_pagina_li_element.get_attribute("class"):
            logger.info("Última página alcançada (botão 'Próximo' <li> tem classe 'disabled').")
            return False

        # Antes de tentar clicar, garanta que está visível e tente centralizá-lo
        logger.debug("Centralizando o botão 'Próximo' na tela.")
        driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", proxima_pagina_li_element)
        time.sleep(0.5) # Pequena pausa para a rolagem e para o DOM assentar

        # TENTATIVA 1: Clicar no elemento <a> dentro do <li>
        try:
            logger.info("Tentativa 1: Clicar no elemento <a> dentro do <li>.")
            # Espera o <a> dentro do <li> ser clicável
            link_interno_a = WebDriverWait(proxima_pagina_li_element, 5).until(
                EC.element_to_be_clickable((By.XPATH, ".//a")) # XPath relativo para o <a>
            )
            link_interno_a.click()
            
            # Sucesso, agora aguarde o carregamento da nova página/tabela
            wait_for_loading_to_disappear(driver) # Sua função de esperar o loader
            WebDriverWait(driver, 15).until( # Espera a tabela ser recarregada
                EC.presence_of_element_located((By.ID, "dataTables-Pagamentos")) 
                # Adicionar uma verificação de que o conteúdo da tabela mudou seria ainda melhor
            )
            logger.info("Navegou para a próxima página (clique no <a> bem-sucedido).")
            return True
        except (ElementClickInterceptedException, TimeoutException, Exception) as e1:
            logger.warning(f"Tentativa 1 (clique no <a>) falhou: {type(e1).__name__} - {e1}. Tentando alternativa.")

        # TENTATIVA 2: Clique JavaScript no elemento <li>
        # Isso pode contornar problemas de interceptação ou eventos não disparados.
        try:
            logger.info("Tentativa 2: Clique via JavaScript no elemento <li>.")
            driver.execute_script("arguments[0].click();", proxima_pagina_li_element)
            
            wait_for_loading_to_disappear(driver)
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.ID, "dataTables-Pagamentos"))
            )
            logger.info("Navegou para a próxima página (clique JavaScript no <li> bem-sucedido).")
            return True
        except Exception as e2:
            logger.error(f"Tentativa 2 (clique JavaScript no <li>) também falhou: {type(e2).__name__} - {e2}")
            return False # Ambas as tentativas principais falharam

    except (TimeoutException, NoSuchElementException):
        logger.info("Botão 'Próximo' (<li>) não encontrado ou não presente/visível inicialmente.")
        return False # Não há mais páginas ou o botão não foi encontrado
    except Exception as e_geral:
        logger.error(f"Erro inesperado ao tentar ir para a próxima página: {str(e_geral)}")
        return False

## 5. Função de orquestração Mensal/Anual

- **extrair_mes_ano(driver, ano: str, mes: str, output_dir: str)**:
    - Esta função orquestra a seleção do ano/mês, chama extrair_dados_pagamentos() repetidamente (enquanto há páginas) e ir_para_proxima_pagina(), e finalmente salva os dados. Ela depende de todas as funções anteriores.

In [ ]:
def extrair_mes_ano(driver, ano: str, mes: str, output_dir: str):
    """
    INCLUIR CIDADE NOS PARAMETROS
    Orquestra a extração de dados para um ano e mês específicos.
    """
    wait_for_page_load(driver)
    logger.info(f" Iniciando extração para {mes}/{ano}")
    try:
        selecionar_ano_mes(driver, ano, mes)
        todos_dados_mes = pd.DataFrame()
        pagina_atual = 1
        while True:
            logger.info(f" Extraindo dados da página {pagina_atual} para {mes}/{ano}...")
            dados_pagina = extrair_dados_pagamentos(driver)
            if not dados_pagina.empty:
                todos_dados_mes = pd.concat([todos_dados_mes, dados_pagina], ignore_index=True)
                logger.info(f" {len(dados_pagina)} registros extraídos da página {pagina_atual}.")
            else:
                logger.info(" Nenhuns dados encontrados nesta página. Passando para a próxima ou finalizando.")

            if not ir_para_proxima_pagina(driver):
                break # Sai do loop se não houver mais páginas
            pagina_atual += 1

        # Salvar os dados do mês em um CSV
        if not todos_dados_mes.empty:
            output_filepath = os.path.join(output_dir, f"royalties_aracaju_{ano}_{mes}.csv")
            todos_dados_mes.to_csv(output_filepath, index=False, encoding='utf-8')
            logger.info(f" Dados de {mes}/{ano} salvos em: {output_filepath}")
        else:
            logger.info(f" Nenhum dado de royalties encontrado para {mes}/{ano}. Nenhum arquivo salvo.")

    except Exception as e:
        logger.error(f" Erro durante a extração para {mes}/{ano}: {e}")
        # Considerar fechar o driver e reiniciar para o próximo mês/ano em caso de erro grave
        # driver.quit()
        # driver = start_driver(headless=False)
        # selecionar_pagamentos(driver) # Reiniciar o estado inicial

## 6. Funções de Fluxo Principal (Sequencial/Paralelo)

- **worker_extrair_mes_ano(ano_mes_tuple, output_dir: str) (para paralelismo)**:
    - Se você for usar extração paralela, esta função será a "tarefa" para cada thread/processo. Ela precisará criar seu próprio driver e chamar selecionar_pagamentos() e extrair_mes_ano().
- **extracao_sequencial(anos: List[str], meses: List[str], output_dir: str)**:
    - Envolve a inicialização de um driver e o loop sobre extrair_mes_ano().
- **extracao_paralela(anos: List[str], meses: List[str], output_dir: str, max_workers: int = 4)**:
    - Orquestra a execução de worker_extrair_mes_ano() em múltiplos threads/processos.

In [ ]:
def worker_extrair_mes_ano(ano_mes_tuple, output_dir: str):
    """
    Função worker para ThreadPoolExecutor ou ProcessPoolExecutor.
    Cada worker terá sua própria instância de driver.
    """
    ano, mes = ano_mes_tuple
    driver_worker = None
    try:
        driver_worker = start_driver(headless=True, url="portal_aracaju") # Headless para paralelismo
        selecionar_pagamentos(driver_worker)
        extrair_mes_ano(driver_worker, ano, mes, output_dir)
    except Exception as e:
        logger.error(f" Erro no worker para {mes}/{ano}: {e}")
    finally:
        if driver_worker:
            driver_worker.quit()

# def extracao_sequencial(anos: List[str], meses: List[str], output_dir: str):
#     """
#     Executa a extração de dados sequencialmente para os anos e meses fornecidos.
#     """
#     driver = None
#     try:
#         driver = start_driver(headless=True, url="portal_cidadao")
#         selecionar_pagamentos(driver) # Isso é executado apenas uma vez no início

#         for ano in anos:
#             for mes in meses:
#                 extrair_mes_ano(driver, ano, mes, output_dir)
#                 # Pequena pausa para evitar sobrecarga no servidor ou para estabilidade
#                 time.sleep(2)
#     except Exception as e:
#         logger.critical(f" Erro fatal na extração sequencial: {e}")
#     finally:
#         if driver:
#             driver.quit()
#             logger.info(" Driver do Chrome fechado.")

def extracao_paralela(anos: List[str], meses: List[str], output_dir: str, max_workers: int = 2):
    """
    Executa a extração de dados em paralelo para os anos e meses fornecidos.
    """
    # Incluir aqui OS.MAKEDIR
    tarefas = [(ano, mes) for ano in anos for mes in meses]
    logger.info(f" Iniciando extração paralela com {max_workers} workers para {len(tarefas)} meses/anos.")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Usando partial para passar output_dir para a função worker
        func_com_args = partial(worker_extrair_mes_ano, output_dir=output_dir)
        futures = [executor.submit(func_com_args, tarefa) for tarefa in tarefas]

        for future in tqdm(as_completed(futures), total=len(futures), desc="Progresso da Extração Paralela"):
            try:
                future.result() # Isso levantará qualquer exceção que ocorreu no worker
            except Exception as e:
                logger.error(f" Tarefa paralela falhou: {e}")
    logger.info(" Extração paralela concluída.")

### Teste 1 - Mês e Ano específicos
Inicializa um novo driver, navega para a seção correta e extrai dados para um único mês/ano.

In [ ]:
# def teste_mes_ano_especifico(ano_teste: str, mes_teste: str, output_dir_teste: str):
#     logger.info(f"Iniciando teste para {mes_teste}/{ano_teste}")
#     driver = None
#     try:
#         # Inicia o driver (portal_cidadao é o padrão e o que parece ser usado)
#         driver = start_driver(headless=True, url="portal_cidadao") # Use headless=True para rodar em background
        
#         # Navega para a seção de pagamentos.
#         selecionar_pagamentos(driver)
        
#         # Extrai os dados para o mês e ano especificados
#         extrair_mes_ano(driver, ano_teste, mes_teste, output_dir_teste)
        
#         logger.info(f"Teste para {mes_teste}/{ano_teste} concluído com sucesso.")
        
#     except Exception as e:
#         logger.error(f"Erro durante o teste para {mes_teste}/{ano_teste}: {e}")
#     finally:
#         if driver:
#             driver.quit()
#             logger.info("Driver do Chrome fechado para teste específico.")

# # Exemplo de uso:
# ANO_ESPECIFICO = "2015"
# MES_ESPECIFICO = "09"
# # Crie um diretório específico para este teste, se desejar, ou use o OUTPUT_DIR principal.
# # os.makedirs(f"{OUTPUT_DIR}/teste_especifico", exist_ok=True)
# # teste_mes_ano_especifico(ANO_ESPECIFICO, MES_ESPECIFICO, f"{OUTPUT_DIR}/teste_especifico")
# # ou para salvar no diretório principal:
# teste_mes_ano_especifico(ANO_ESPECIFICO, MES_ESPECIFICO, OUTPUT_DIR)

### Teste 2 - Ano específico com todos os meses
Itera por todos os meses de um ano específico, usando uma única instância de driver para o ano.

In [ ]:
# def teste_ano_especifico_todos_meses(ano_teste: str, meses_teste: List[str], output_dir_teste: str):
#     logger.info(f"Iniciando teste para o ano {ano_teste} (todos os meses)")
#     driver = None
#     try:
#         driver = start_driver(headless=True, url="portal_cidadao")
#         selecionar_pagamentos(driver) # Navega para pagamentos uma vez

#         for mes_teste in meses_teste:
#             logger.info(f"Iniciando extração para {mes_teste}/{ano_teste}")
#             extrair_mes_ano(driver, ano_teste, mes_teste, output_dir_teste)
#             time.sleep(2) # Pequena pausa entre os meses

#         logger.info(f"Teste para o ano {ano_teste} concluído com sucesso.")

#     except Exception as e:
#         logger.error(f"Erro durante o teste para o ano {ano_teste}: {e}")
#     finally:
#         if driver:
#             driver.quit()
#             logger.info(f"Driver do Chrome fechado para teste do ano {ano_teste}.")

# # Exemplo de uso:
# ANO_PARA_TESTAR_COMPLETO = "2015"
# # os.makedirs(f"{OUTPUT_DIR}/teste_ano_{ANO_PARA_TESTAR_COMPLETO}", exist_ok=True)
# # teste_ano_especifico_todos_meses(ANO_PARA_TESTAR_COMPLETO, MESES, f"{OUTPUT_DIR}/teste_ano_{ANO_PARA_TESTAR_COMPLETO}")
# # ou para salvar no diretório principal:
# teste_ano_especifico_todos_meses(ANO_PARA_TESTAR_COMPLETO, MESES, OUTPUT_DIR)

### Teste 3 - Teste com vários anos (SEQUENCIAL)
Usar a função ``extracao_sequencial`` diretamente, fornecendo uma lista personalizada de anos.

In [ ]:
# # Exemplo de uso para vários anos específicos (e todos os meses para esses anos):
# ANOS_DE_TESTE_SEQUENCIAL = ["2020", "2021"] # Lista de anos que você quer testar
# # os.makedirs(f"{OUTPUT_DIR}/teste_sequencial_varios_anos", exist_ok=True)
# # logger.info(f"Iniciando extração sequencial para os anos: {ANOS_DE_TESTE_SEQUENCIAL}")
# # extracao_sequencial(ANOS_DE_TESTE_SEQUENCIAL, MESES, f"{OUTPUT_DIR}/teste_sequencial_varios_anos")
# # ou para salvar no diretório principal:
# extracao_sequencial(ANOS_DE_TESTE_SEQUENCIAL, MESES, OUTPUT_DIR)

### 4. Execução Sequencial para Todos os Anos e Meses Definidos
Utilize a função ``extracao_sequencial`` com as constantes ``ANOS`` e ``MESES``.

In [ ]:
# logger.info("Iniciando extração sequencial completa...")
# # ANOS e MESES são as constantes definidas no início do seu notebook
# # OUTPUT_DIR também é a constante definida
# extracao_sequencial(ANOS, MESES, OUTPUT_DIR)
# logger.info("Extração sequencial completa finalizada.")

### 5. Execução Paralela para Todos os Anos e Meses Definidos
Utilize a função ``extracao_paralela``. Você pode ajustar ``max_workers``.

In [ ]:
ANOS_PARA_EXTRAIR = [str(ano) for ano in range(2020,2024)] # Exemplo: 2022, 2023, 2024
MESES_PARA_EXTRAIR = [f"{m:02d}" for m in range(1, 13)] # Todos os meses
OUTPUT_DIR =f'../data/aracaju/2020_2024_dados_royalties_aracaju'
os.makedirs(OUTPUT_DIR, exist_ok=True)  # Cria o diretório se não existir

logger.info("Iniciando o processo de extração de dados de royalties.")

# Opção 2: Extração Paralela
extracao_paralela(ANOS_PARA_EXTRAIR, MESES_PARA_EXTRAIR, OUTPUT_DIR,max_workers=4)

### CRIAÇÃO DE CSV CONSOLIDADO POR ANO

In [25]:
import pandas as pd
import os
import glob

def unir_royalties_por_ano(caminho_da_pasta, ano, separador=';'):
    """
    Busca arquivos no formato 'prefixo_ano_mes.csv', os une em um único arquivo
    e o salva com o nome 'prefixo_ano_consolidado.csv'.

    Args:
        caminho_da_pasta (str): O caminho para a pasta contendo os arquivos CSV.
        ano (int): O ano dos arquivos a serem processados (ex: 2015).
        separador (str): O separador utilizado nos arquivos CSV (padrão: ',').

    Returns:
        pandas.DataFrame: Um DataFrame com todos os dados unidos para o ano especificado,
                          ou None se nenhum arquivo for encontrado.
    """
    # Valida se o caminho existe
    if not os.path.isdir(caminho_da_pasta):
        logger.error(f"❌ Erro: O caminho não existe ou não é uma pasta: '{caminho_da_pasta}'")
        return None

    # Cria um padrão de busca específico para o ano, ex: '*_2015_??.csv'
    # O '??' funciona como um coringa para os dois dígitos do mês.
    padrao_busca = os.path.join(caminho_da_pasta, f"*_{ano}_??.csv")
    lista_de_arquivos = glob.glob(padrao_busca)

    # Verifica se encontrou arquivos para o ano especificado
    if not lista_de_arquivos:
        logger.error(f"⚠️ Aviso: Nenhum arquivo encontrado para o ano de {ano} no padrão 'nome_{ano}_mes.csv'")
        logger.error(f"   (Busca realizada em: {caminho_da_pasta})")
        return None
    
    # --- Geração automática do nome do arquivo de saída ---
    # Pega o nome do primeiro arquivo encontrado para extrair o prefixo
    primeiro_arquivo = os.path.basename(lista_de_arquivos[0])
    # Divide o nome do arquivo por '_' e remove as duas últimas partes (ano e mês)
    prefixo_nome = '_'.join(primeiro_arquivo.split('_')[:-2])
    # Cria o nome do arquivo de saída
    nome_arquivo_saida = f"{prefixo_nome}_{ano}_consolidado.csv"
    
    logger.info(f"Nome do arquivo de saída será: '{nome_arquivo_saida}'")

    # Lista para armazenar os DataFrames
    lista_de_dataframes = []
    
    logger.debug(f"Lendo {len(lista_de_arquivos)} arquivo(s) para o ano de {ano}:")
    for arquivo in sorted(lista_de_arquivos): # 'sorted()' garante a ordem (mês 01, 02, ...)
        print(f"  - {os.path.basename(arquivo)}")
        try:
            df_mensal = pd.read_csv(arquivo, sep=separador)
            lista_de_dataframes.append(df_mensal)
        except Exception as e:
            logger.error(f"  ❌ Erro ao ler o arquivo {os.path.basename(arquivo)}: {e}")

    if not lista_de_dataframes:
        logger.info("⚠️ Nenhum arquivo foi lido com sucesso.")
        return None

    # Concatena todos os DataFrames
    df_consolidado = pd.concat(lista_de_dataframes, ignore_index=True)

    # Salva o resultado
    caminho_saida = os.path.join(caminho_da_pasta, nome_arquivo_saida)
    df_consolidado.to_csv(caminho_saida, index=False, encoding='utf-8-sig')

    logger.info("\n-------------------------------------------")
    logger.info("✅ Sucesso! Arquivos combinados.")
    logger.info(f"   Total de linhas no novo arquivo: {len(df_consolidado)}")
    logger.info(f"   Salvo em: {caminho_saida}")

    return df_consolidado

In [26]:
# --- 1. Defina a pasta e o ano que deseja processar ---
pasta_dos_meus_dados = f'../data/aracaju/2024_dados_royalties_aracaju'
ano_para_processar = 2024


# --- 2. Chame a função passando a pasta e o ano ---
# A função irá encontrar os arquivos de 2015 e criar, por exemplo,
# 'royalties_aracaju_2015_consolidado.csv' automaticamente.
df_ano_consolidado = unir_royalties_por_ano(pasta_dos_meus_dados, ano_para_processar)


# --- 3. Verifique o resultado ---
if df_ano_consolidado is not None:
    # Mostra as 5 primeiras e as 5 últimas linhas para verificar a junção
    print("\nInício do DataFrame consolidado:")
    display(df_ano_consolidado.head())
    
    print("\nFinal do DataFrame consolidado:")
    display(df_ano_consolidado.tail())

[MainThread] - 2025-07-17 08:11:38,197 - INFO - Nome do arquivo de saída será: 'aracaju_royalties_2024_consolidado.csv'
[MainThread] - 2025-07-17 08:11:38,241 - INFO - 
-------------------------------------------
[MainThread] - 2025-07-17 08:11:38,243 - INFO - ✅ Sucesso! Arquivos combinados.
[MainThread] - 2025-07-17 08:11:38,244 - INFO -    Total de linhas no novo arquivo: 158
[MainThread] - 2025-07-17 08:11:38,245 - INFO -    Salvo em: ../data/aracaju/2024_dados_royalties_aracaju\aracaju_royalties_2024_consolidado.csv


  - aracaju_royalties_2024_01.csv
  - aracaju_royalties_2024_02.csv
  - aracaju_royalties_2024_04.csv
  - aracaju_royalties_2024_05.csv
  - aracaju_royalties_2024_06.csv
  - aracaju_royalties_2024_07.csv
  - aracaju_royalties_2024_08.csv
  - aracaju_royalties_2024_09.csv
  - aracaju_royalties_2024_11.csv
  - aracaju_royalties_2024_12.csv

Início do DataFrame consolidado:


,orgao,unidade,data,empenho,processo,credor,cpf_cnpj,pago,retido,anulacao,...,elemento,subelemento,fonte_de_recurso,conta_banco,tipo,no_doc,data_atesto,responsavel_atesto,historico_empenho,historico_pagamento
0,13 - SECRETARIA MUNICIPAL DA FAZENDA,13101 - SECRETARIA MUNICIPAL DA FAZENDA,31/01/2024,126002,131024,DELEGACIA DA RECEITA FEDERAL DO BRASIL EM ARACAJU,00.394.460/0092-89,"R$ 7.482,53","R$ 0,00","R$ 0,00",...,33904700 - Obrigações Tributárias e Contributivas,33904712 - Contribuicao Para O Pis ou Pasep,17200000 - Transferências da União Referentes ...,Bc: 104 / Ag: 59 / CC: 4988,Autorização de Débito,41,NaN,NaN,VALOR PARA COBRIR DESPESAS COM OBRIGAÇÕES TRIB...,VALOR PARA COBRIR DESPESAS COM OBRIGAÇÕES TRIB...
1,13 - SECRETARIA MUNICIPAL DA FAZENDA,13101 - SECRETARIA MUNICIPAL DA FAZENDA,31/01/2024,126002,131025,DELEGACIA DA RECEITA FEDERAL DO BRASIL EM ARACAJU,00.394.460/0092-89,"R$ 6.036,49","R$ 0,00","R$ 0,00",...,33904700 - Obrigações Tributárias e Contributivas,33904712 - Contribuicao Para O Pis ou Pasep,17200000 - Transferências da União Referentes ...,Bc: 104 / Ag: 59 / CC: 4988,Autorização de Débito,41,NaN,NaN,VALOR PARA COBRIR DESPESAS COM OBRIGAÇÕES TRIB...,VALOR PARA COBRIR DESPESAS COM OBRIGAÇÕES TRIB...
2,27 - SECRETARIA MUNICIPAL DA INFRAESTRUTURA,27301 - EMPRESA MUNICIPAL DE OBRAS E URBANIZAÇ...,30/01/2024,124001,130015,STRATURA ASFALTOS LTDA.,59.128.553/0025-44,"R$ 178.530,84","R$ 0,00","R$ 0,00",...,33909200 - Despesas de Exercícios Anteriores,33909230 - Material de Consumo,17200000 - Transferências da União Referentes ...,Bc: 104 / Ag: 2175 / CC: 34587,Autorização de Débito,146,26/12/2023,THIAGO JOSÉ RAMOS DOS SANTOS,"AQUISIÇÃO DE PRODUTOS ASFALTICOS, CAP, CONFORM...",LIQUIDAÇÃO DO BM Nº 18 (22/12/2023) REF. AQUIS...
3,27 - SECRETARIA MUNICIPAL DA INFRAESTRUTURA,27301 - EMPRESA MUNICIPAL DE OBRAS E URBANIZAÇ...,30/01/2024,122006,130020,BETUNEL INDUSTRIA E COMERCIO S/A,60.546.801/0004-21,"R$ 12.680,40","R$ 0,00","R$ 0,00",...,33909200 - Despesas de Exercícios Anteriores,33909230 - Material de Consumo,17200000 - Transferências da União Referentes ...,Bc: 104 / Ag: 2175 / CC: 34587,Autorização de Débito,148,16/01/2024,VANDERSON PEREIRA SANTOS SOARES,"AQUISIÇÃO DE PRODUTOS ASFALTICOS, SENDO: CAP, ...",LIQUIDAÇÃO DO BM Nº 123 (28/11/2023 A 26/12/20...
4,27 - SECRETARIA MUNICIPAL DA INFRAESTRUTURA,27301 - EMPRESA MUNICIPAL DE OBRAS E URBANIZAÇ...,30/01/2024,122006,130021,BETUNEL INDUSTRIA E COMERCIO S/A,60.546.801/0004-21,"R$ 11.790,39","R$ 0,00","R$ 0,00",...,33909200 - Despesas de Exercícios Anteriores,33909230 - Material de Consumo,17200000 - Transferências da União Referentes ...,Bc: 104 / Ag: 2175 / CC: 34587,Autorização de Débito,148,16/01/2024,VANDERSON PEREIRA SANTOS SOARES,"AQUISIÇÃO DE PRODUTOS ASFALTICOS, SENDO: CAP, ...",LIQUIDAÇÃO DO BM Nº 123 (28/11/2023 A 26/12/20...



Final do DataFrame consolidado:


,orgao,unidade,data,empenho,processo,credor,cpf_cnpj,pago,retido,anulacao,...,elemento,subelemento,fonte_de_recurso,conta_banco,tipo,no_doc,data_atesto,responsavel_atesto,historico_empenho,historico_pagamento
153,27 - SECRETARIA MUNICIPAL DA INFRAESTRUTURA,27301 - EMPRESA MUNICIPAL DE OBRAS E URBANIZAÇ...,03/12/2024,116008,1203061,STRATURA ASFALTOS S.A.,59.128.553/0029-78,"R$ 1.026,66","R$ 0,00","R$ 0,00",...,33903000 - Material de Consumo,33903033 - Material Para Produção Industrial,17050000 - Transferência dos Estados Referente...,Bc: 104 / Ag: 2175 / CC: 34587,Autorização de Débito,3041,12/11/2024,THIAGO JOSÉ RAMOS DOS SANTOS,"AQUISIÇÃO DE PRODUTOS ASFALTICOS, SENDO: CAP, ...",LIQUIDAÇÃO DO BM Nº 77 (01/11/2024 A 07/11/202...
154,27 - SECRETARIA MUNICIPAL DA INFRAESTRUTURA,27301 - EMPRESA MUNICIPAL DE OBRAS E URBANIZAÇ...,03/12/2024,116008,1203062,STRATURA ASFALTOS S.A.,59.128.553/0029-78,"R$ 1.026,66","R$ 0,00","R$ 0,00",...,33903000 - Material de Consumo,33903033 - Material Para Produção Industrial,17050000 - Transferência dos Estados Referente...,Bc: 104 / Ag: 2175 / CC: 34587,Autorização de Débito,3041,18/11/2024,THIAGO JOSÉ RAMOS DOS SANTOS,"AQUISIÇÃO DE PRODUTOS ASFALTICOS, SENDO: CAP, ...",LIQUIDAÇÃO DO BM Nº 78 (08/11/2024 A 13/11/202...
155,27 - SECRETARIA MUNICIPAL DA INFRAESTRUTURA,27301 - EMPRESA MUNICIPAL DE OBRAS E URBANIZAÇ...,03/12/2024,116008,1203056,STRATURA ASFALTOS S.A.,59.128.553/0029-78,"R$ 189.456,84","R$ 0,00","R$ 0,00",...,33903000 - Material de Consumo,33903033 - Material Para Produção Industrial,17050000 - Transferência dos Estados Referente...,Bc: 104 / Ag: 2175 / CC: 34587,Autorização de Débito,3038,14/10/2024,THIAGO JOSÉ RAMOS DOS SANTOS,"AQUISIÇÃO DE PRODUTOS ASFALTICOS, SENDO: CAP, ...",LIQUIDAÇÃO DO BM Nº 72 (05/10/2024 A 11/10/202...
156,27 - SECRETARIA MUNICIPAL DA INFRAESTRUTURA,27301 - EMPRESA MUNICIPAL DE OBRAS E URBANIZAÇ...,03/12/2024,102267,1203063,STRATURA ASFALTOS S.A.,59.128.553/0021-10,"R$ 202.240,26","R$ 0,00","R$ 0,00",...,33903000 - Material de Consumo,33903033 - Material Para Produção Industrial,17200000 - Transferências da União Referentes ...,Bc: 104 / Ag: 2175 / CC: 34587,Autorização de Débito,3037,28/11/2024,THIAGO JOSÉ RAMOS DOS SANTOS,"AQUISIÇÃO DE PRODUTOS ASFALTICOS, SENDO: CAP, ...",LQUIDAÇÃO DO BM Nº 75 (18/10/2024 A 23/10/2024...
157,27 - SECRETARIA MUNICIPAL DA INFRAESTRUTURA,27301 - EMPRESA MUNICIPAL DE OBRAS E URBANIZAÇ...,03/12/2024,102267,1203065,STRATURA ASFALTOS S.A.,59.128.553/0021-10,"R$ 185.031,81","R$ 0,00","R$ 0,00",...,33903000 - Material de Consumo,33903033 - Material Para Produção Industrial,17200000 - Transferências da União Referentes ...,Bc: 104 / Ag: 2175 / CC: 34587,Autorização de Débito,3040,28/10/2024,THIAGO JOSÉ RAMOS DOS SANTOS,"AQUISIÇÃO DE PRODUTOS ASFALTICOS, SENDO: CAP, ...",LQUIDAÇÃO DO BM Nº 75 (18/10/2024 A 23/10/2024...


## 7. Execução

### 7.1 Extração Sequencial

In [ ]:
# 1. Definição das constantes (ANOS, MESES, COLUNAS, OUTPUT_DIR)
# 2. Configuração do Logging
# 3. Definição de todas as funções na ordem listada acima.

# if __name__ == "__main__":
#     ANOS_PARA_EXTRAIR = [str(ano) for ano in range(2022, 2024)] # Exemplo: 2022 e 2023
#     MESES_PARA_EXTRAIR = [f"{m:02d}" for m in range(1, 4)] # Exemplo: Jan, Fev, Mar

#     logger.info("Iniciando o processo de extração de dados de royalties.")

#     # Opção 1: Extração Sequencial
#     extracao_sequencial(ANOS_PARA_EXTRAIR, MESES_PARA_EXTRAIR, OUTPUT_DIR)
#     logger.info("Extração sequencial concluída.")

### 7.2 Extração Paralela

In [ ]:
# # 1. Definição das constantes (ANOS, MESES, COLUNAS, OUTPUT_DIR)
# # 2. Configuração do Logging
# # 3. Definição de todas as funções na ordem listada acima.

# if __name__ == "__main__":
#     ANOS_PARA_EXTRAIR = [str(ano) for ano in range(2022, 2025)] # Exemplo: 2022, 2023, 2024
#     MESES_PARA_EXTRAIR = [f"{m:02d}" for m in range(1, 13)] # Todos os meses

#     logger.info("Iniciando o processo de extração de dados de royalties.")

#     # Opção 2: Extração Paralela
#     extracao_paralela(ANOS_PARA_EXTRAIR, MESES_PARA_EXTRAIR, OUTPUT_DIR, max_workers=2)
#     logger.info("Extração paralela concluída.")

### FIM =)